# n-puzzle problem

In [115]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import functools

In [116]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

## Utility functions

In [117]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper

In [ ]:
class State :
    def __init__(self, matrix: np.ndarray):
        self.matrix = matrix
        self.g = float('inf') #cost root to current
        self.f = float('inf') #cost root to goal
        self.h = float('inf') #heuristic : estimated cost from current to goal

    #check if the current state contains a matrix that represents the solution
    def is_goal(self):
        return np.array_equal(self.matrix, np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)))
   
    def is_solvable(self) -> bool:
        inv_count = 0
        tmp_state = self.matrix.copy().reshape(PUZZLE_DIM**2)

        # Count inversions
        for i in range(0, PUZZLE_DIM**2):
            for j in range(i + 1, PUZZLE_DIM**2):
                if tmp_state[j] != 0 and tmp_state[i] != 0 and tmp_state[i] > tmp_state[j]:
                    inv_count += 1

        if PUZZLE_DIM % 2 != 0:  # Odd grid size
            return inv_count % 2 == 0
        else:  # Even grid size
            blank_row = PUZZLE_DIM - np.where(self.matrix == 0)[0][0] # Row of blank (from bottom)
        return (inv_count + blank_row) % 2 != 0

In [201]:
def cost_root_to_current(path : list):
    return len(path)

def manhattan_heuristic(state:State,goal_state:State)->int:
    manhattan = 0
    for elem in range(1,PUZZLE_DIM**2):
        start_coords = np.where(state.matrix == elem)
        end_coords = np.where(goal_state.matrix == elem)
        manhattan+=  abs(start_coords[0]-end_coords[0])+abs(start_coords[1]-end_coords[1])
    
    return manhattan
    # linear_conflict_penalty = 0

    # for i in range(PUZZLE_DIM):
    #     # Row conflicts
    #     goal_row = {np.int64(goal_state.matrix[i][j]): j for j in range(PUZZLE_DIM)}
    #     row = state.matrix[i]
    #     for j in range(PUZZLE_DIM):
    #         for k in range(j + 1, PUZZLE_DIM):
    #             if row[j] != 0 and row[k] != 0 and goal_row[row[j]] > goal_row[row[k]]:
    #                 linear_conflict_penalty += 2

    #     # Column conflicts
    #     goal_col = {np.int64(goal_state.matrix[j][i]): j for j in range(PUZZLE_DIM)}
    #     col = [state.matrix[j][i] for j in range(PUZZLE_DIM)]
    #     for j in range(PUZZLE_DIM):
    #         for k in range(j + 1, PUZZLE_DIM):
    #             if col[j] != 0 and col[k] != 0 and goal_col[col[j]] > goal_col[col[k]]:
    #                 linear_conflict_penalty += 2

    # return manhattan + linear_conflict_penalty


class WalkingDistance:
    def __init__(self, puzzle_dim):
        self.puzzle_dim = puzzle_dim
        self.row_table = self._precompute_tables(is_row=True)
        self.col_table = self._precompute_tables(is_row=False)

    def _precompute_tables(self, is_row):
        """
        Precompute the walking distance tables for rows or columns.
        Each entry in the table represents the number of moves required for a tile
        to move from one position to another in the same row or column.
        """
        table = {}
        for tile in range(1, self.puzzle_dim**2):  # Exclude the blank tile (0)
            table[tile] = self._precompute_tile_distances(tile, is_row)
        return table

    def _precompute_tile_distances(self, tile, is_row):
        """
        Precompute distances for a specific tile in either rows or columns.
        """
        table = {}
        for position in range(self.puzzle_dim):
            distances = [0] * self.puzzle_dim
            for target in range(self.puzzle_dim):
                distances[target] = abs(target - position)
            table[position] = distances
        return table

    def calculate(self, state: State, goal_state: State) -> int:
        """
        Compute the walking distance heuristic for a given state and goal state.
        """
        wd = 0

        # Calculate row conflicts
        for row in range(self.puzzle_dim):
            wd += self._calculate_conflict(state, goal_state, row, is_row=True)

        # Calculate column conflicts
        for col in range(self.puzzle_dim):
            wd += self._calculate_conflict(state, goal_state, col, is_row=False)

        return wd

    def _calculate_conflict(self, state: State, goal_state: State, index: int, is_row: bool) -> int:
        """
        Calculate the conflicts for a specific row or column.
        """
        # Extract the row or column tiles
        if is_row:
            tiles = state.matrix[index, :]
            goal_tiles = goal_state.matrix[index, :]
        else:
            tiles = state.matrix[:, index]
            goal_tiles = goal_state.matrix[:, index]

        # Exclude the blank tile (0)
        tiles = tiles[tiles != 0]
        goal_tiles = goal_tiles[goal_tiles != 0]

        # Calculate walking distance for the current row or column
        distance = 0
        for tile in tiles:
            current_pos = np.where(state.matrix == tile)
            target_pos = np.where(goal_state.matrix == tile)

            if is_row:
                current_idx = current_pos[1][0]  # Column index
                target_idx = target_pos[1][0]
                distance += self.row_table[tile][current_idx][target_idx]
            else:
                current_idx = current_pos[0][0]  # Row index
                target_idx = target_pos[0][0]
                distance += self.col_table[tile][current_idx][target_idx]

        return distance


def available_actions(state: State) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state.matrix == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions


@counter
def do_action(state: State, action: 'Action') -> State:
    new_state = state.matrix.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return State(new_state)

def print_solution(solution : tuple[list,int]):
    if solution[0] == None:
        print("No solution was found")
    else:
        print(f"Solution was found in {len(solution[0])} steps with cost of {do_action.calls}")
        print("Solution path:")
        for state in solution[0]:
            print(state.matrix)
            print()


## IDA*

In [120]:
# takes as parameter a state and a heuristic function
def ida_star(initial_state : State, goal_state : State, heuristic):
    bound = heuristic(initial_state,goal_state)
    path = [initial_state]

    while True:
        t = search(path, 0, bound, heuristic, goal_state, set())
        if t < 0: ## FOUND = -1
            return (path, bound)
        elif t == float('inf'):
            return (None, float('inf')) 
        else: # t belongs to [0,inf[
            bound = t
           # path.push(t)

def search(path : list, g : float, bound : int, h, goal_state : State, visited = set()) :
    node = path[-1]
    f = g + h(node, goal_state)
    if f > bound :
        return f
    if node.is_goal():
        return -1 # FOUND
    
    min = float('inf')
    visited.add(node)

    successors = [do_action(node, action) for action in available_actions(node)]

    for successor in sorted(successors, key=lambda s: h(s, goal_state)):
        if successor not in visited:
            path.append(successor)
            t = search(path, g + 1, bound, h, goal_state, visited)
            if t == -1:
                return -1
            if t < min :
                min = t
            path.pop()


    visited.remove(node) #backtrack
    return min

### simple 3x3 test

In [121]:
#state = State(np.array([np.array([1,8,2]),np.array([0,4,3]),np.array([7,6,5])]))
state = State(np.array([np.array([0,2,1]),np.array([3,7,5]),np.array([8,6,4])]))
print(state.is_solvable())

ValueError: cannot reshape array of size 9 into shape (16,)

### complex 5x5 test

In [ ]:
state = np.array([[13,  9,  0, 10, 19],
                [ 3, 21, 14,  5,  8],
                [22, 16,  4, 24, 18],
                [ 6,  2, 11,  1, 20],
                [ 7, 15,23,12,17]])

state = State(state)
print(state.is_solvable())
print(is_solvable(state.matrix))

True
True


###  4x4 test

In [202]:
state = np.array([[14,  13,  15, 8],
                [ 1, 4 , 12, 6],
                [2, 7,  10, 3],
                [ 11,  5, 0,  9]])

state = State(state)
print(state.is_solvable())

1
64
True


In [162]:
state = np.array([[12,  1,  2, 15],
                [ 11, 6 , 5,8],
                [7, 10,  9, 4],
                [ 0,  13, 14,  3]])

state = State(state)
print(state.is_solvable())

1
48
True


### random state

In [197]:
RANDOMIZE_STEPS = 250
solvable = False
state = State(np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)))

# while not solvable:

for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

    # solvable = is_solvable(state.matrix)
print(state.is_solvable())
print(state.matrix)

Randomizing:   0%|          | 0/250 [00:00<?, ?it/s]

4
55
True
[[13  7  0  5]
 [ 6  9 14  4]
 [11  2 15  1]
 [10  3  8 12]]


### find solution

In [ ]:
wd_heuristic = WalkingDistance(PUZZLE_DIM)

print(state.matrix)
goal_state = State(np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)))

if  (state.is_solvable()):
    #solution = ida_star(state,goal_state, wd_heuristic.calculate)
    solution = ida_star(state,goal_state, manhattan_heuristic)
    print_solution(solution)
else:
    print("not Solvable")

[[14 13 15  8]
 [ 1  4 12  6]
 [ 2  7 10  3]
 [11  5  0  9]]
1
64
